# Readme file for Eva's local changes to CTSM folder to get COSMOREA forcing data working

Started 2023-02-28

Instructions copied from @huitang-earth's [scripts_ctsm_region/atm_forcing/cosmo_rea_6km](https://github.com/huitang-earth/scripts_ctsm_region/tree/main/atm_forcing/cosmo_rea_6km) and @ecaas, with additions and modifications by @evalieungh. 

As explained in [Hui's readme file](https://github.com/huitang-earth/scripts_ctsm_region/tree/main/atm_forcing/cosmo_rea_6km), COSMA data are available from <https://opendata.dwd.de/climate_environment/REA/COSMO_REA6/>. Single-site forcing for the VCG sites, subset from the COMSOREA data, is already prepared by Hui on Fram and Saga. Elin downloaded a local copy which is used here. 

Elin's create case command on Saga:
```
/cluster/home/ecaas/CTSM_ecaas_fork/cime/scripts/create_newcase --compset 2000_DATM%COSMOREA_CLM51%BGC_SICE_SOCN_MOSART_SGLC_SWAV --machine saga --run-unsupported --user-mods-dirs /cluster/home/ecaas/ALP1/user_mods --handle-preexisting-dirs r --project nn2806k --case cases/ALP1_I2000.CLM51.COSMOREA.BGC --res CLM_USRDAT
```

To use COSMOREA data, replace the following files with the files provided in "ctsm_config_reg" (works for both regional and any single site simulations):

```
ctsm/components/cdeps/datm/cime_config/namelist_definition_datm.xml
ctsm/components/cdeps/datm/cime_config/stream_definition_datm.xml
ctsm/components/cdeps/datm/cime_config/config_component.xml
```

To use single-site COSMOREA data for VCG, replace the above files with the files provided in "ctsm_config_VCG" (This works only for VCG sites). To better track what has been modified, compare the files with that in "ctsm_config_default".

### Make the changes on local NREC LSP repo

Log into NREC virtual machine (see [issue #156](https://github.com/NorESMhub/noresm-land-sites-platform/issues/156) for VM setup).

Create new branch as copy of existing FATES_INCLINE branch

Download modified namelist, stream, and config files from huitang-earth/scripts_ctsm_region after checking the default files are in place and changing their name by adding "_default".

Open both versions of the files to check that the format is the same, and to see differences.

Next, modify the compset. Elin used `2000_DATM%COSMOREA_CLM51%BGC_SICE_SOCN_MOSART_SGLC_SWAV`, but I will try to replace BGC with FATES first since the LSP is set up to run FATES. Hui's modifications of the files copied in above, should enable a COSMOREA instead of GSWP3 compset by specifying how the COSMO data is organised.

In the LSP (v1), the compset is hardcoded in to resources/config/sites.json and must be changed for each site. This is also where the input data source is specified. 

Open the file in vim and change both compset and input data source, e.g. change compset from `2000_DATM%GSWP3v1_CLM51%FATES_SICE_SOCN_MOSART_SGLC_SWAV` to `2000_DATM%COSMOREA_CLM51%FATES_SICE_SOCN_MOSART_SGLC_SWAV` and the input data from `https://raw.githubusercontent.com/NorESMhub/noresm-lsp-data/main/sites/ALP1.zip` to `https://raw.githubusercontent.com/evalieungh/FATES_INCLINE/main/data/ALP1_cosmo.zip`

Check that the create_newcase command has or can have the setup as Elin. In the LSP, create_newcase is invoked through the API, in NorESMhub/ctsm-api/app/tasks/cases.py. Here is an excerpt from the script:

```
    create_new_case_cmd = [
        str(settings.MODEL_ROOT / "cime" / "scripts" / "create_newcase"),
        "--case",
        str(case_path),
        "--compset",
        case.compset,
        "--driver",
        case.driver,
        "--res",
        "CLM_USRDAT",
        "--machine",
        settings.MACHINE_NAME,
        "--run-unsupported",
        "--handle-preexisting-dirs",
        "r",
    ]

    if (case_data_root / "user_mods").exists():
        create_new_case_cmd.extend(
            [
                "--user-mods-dirs",
                str(case_data_root / "user_mods"),
            ]
        )
```

So it looks like it's equivalent to writing something like this: `./create_newcase --case caseNameFromGUI --compset compsetFromSitesJson --driver NUOPC --res CLM_USRDAT --machine LSPDockerContainer --run-unsuppoerted --handle-preexisting-dirs r --user-mods-dirs resources/data/caseNameFromGUI/user_mods`